# Getting Started with QuickTake

How to call the QuickTake class and use the methods/APIs.

Once instantiated all methods are available to use. 

*_Args_*

The `new_init` argument is used to either instantiate a new model or use an existing model if found in the cache.

---
```
: 24 Aug 2023
: zach wolpe
: zachcolinwolpe@gmail.com
```
---

Instance of `QuickTake` class.

In [ ]:
from IPython.display import display
from PIL import Image
import cv2

# example images
img = './data/random/IMG_5993.jpeg'
img = './data/random/dave_cen.png'
img = './data/random/dave.png'

# to avoid distractions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# int module
from quicktake import QuickTake
qt = QuickTake()

---
# YoloV5: Object Detection
----

In [ ]:
# load and tranform if required
img_        = qt.read_image(img)
transform_  = qt.inference_transforms(input_size_=64)
transform_(img_).shape

In [ ]:
# raw pixels
results_, time_ = qt.yolov5(image=img_, new_init=True)
results_, time_

In [ ]:
# Inference
results_.print() # Results, see: results.save(), help(results) # or .show()
points = results_.pandas().xyxy[0] # get raw results data (points of interest)
print('\n\nObjects detected:\n', points.to_markdown(), '\n\n') # print formatted results
results_.show()

---
# Gender & Age Determination
----

Gender and age determination are trained on faces. They work fine on a larger image, however the will fail to make multiple predictions in the case of multiple faces in a single image.

The APIs intended use is to chain models, thus the `gender` and `age` models are designed to take the output of the `yolo` model as input.

See the source code of `QuickTake.launchStream()` if a more detailed example is required.


In [ ]:
# Fit Yolo to extract faces
qt      = QuickTake()
frame   = qt.read_image(img)

In [ ]:
# generate points
for _label, x0,y0,x1,y1, colour, thickness, results, res_df, age_, gender_ in qt.yolo_loop(frame):
    _label = QuickTake.generate_yolo_label(_label)
    QuickTake.add_block_to_image(frame, _label, x0,y0,x1,y1, colour=colour, thickness=thickness)

In [ ]:
from IPython.display import display
from PIL import Image
import cv2

# outside of notebook
# cv2.imshow("test", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# inside of notebook
#   - In Pillow, the order of colors is assumed to be RGB (red, green, blue).
#   - As we are using Image.fromarray() of PIL module, we need to convert BGR to RGB.

img_view = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
display(Image.fromarray(img_view))

----
# Webcam Stream
----

Launch stream
- best not in a notebook is an interactive terminal is required to kill the session.
- kill the session with 'q' or 'esc' key or 'ctrl + c' in the terminal.

In [ ]:
from quicktake import QuickTake
qt = QuickTake()

In [ ]:
# run
qt.launchStream()

----
# Individual Model Calls
----

Finally, the `QuickTake` class can be used to call each model individually.

_*Note*_: Each model returns the results `results_` as well as the runtime `time_`.

Example inputs parameters:


In [ ]:
image_size  = 64
X           = torch.randn(3, 224, 224, requires_grad=False)#.to(tgp.device)
X_yolo      = torch.randn(1, 3, 224, 224, requires_grad=False)#.to(tgp.device)
image_path  = './data/random/dave.png'
image_paths = './data/random/'

Process a torch.Tensor:

In [ ]:
results_, time_ = QL.gender(X, new_init=True)
results_, time_ = QL.yolov5(X_yolo, new_init=True)
results_, time_ = QL.age(X, new_init=True)

Process a single image (path):

In [ ]:
results_, time_ = QL.gender(image_path, new_init=True)
results_, time_ = QL.yolov5(image_path, new_init=True)
results_, time_ = QL.age(image_path, new_init=True)

Process a directory of images:

In [ ]:
results_, time_ = QL.gender(image_paths, new_init=True)
results_, time_ = QL.yolov5(image_paths, new_init=True)
results_, time_ = QL.age(image_paths, new_init=True)